In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

### Data formats
"ratings.dat" file format - UserID::MovieID::Rating::Timestamp
- UserIDs range between 1 and 6040 
- MovieIDs range between 1 and 3952
- Ratings are made on a 5-star scale (whole-star ratings only)
- Timestamp is represented in seconds since the epoch as returned by time(2)
- Each user has at least 20 ratings


"movies.dat" format - MovieID::Title::Genres
- Genres are pipe-separated and are selected from the following genres: [Action, Adventure, Animation, Children's, Comedy, Crime, Documentary, Drama, Fantasy, Film-Noir, Horror, Musical, Mystery, Romance, Sci-Fi, Thriller, War, Western]
- Titles are identical to titles provided by the IMDB (including
year of release)
- Some MovieIDs do not correspond to a movie due to accidental duplicate
entries and/or test entries
- Movies are mostly entered by hand, so errors and inconsistencies may exist


"users.dat" file format - UserID::Gender::Age::Occupation::Zip-code
- Gender is denoted by a "M" for male and "F" for female
- Age is chosen from the following ranges: {1:  "Under 18", 18:  "18-24", 25:  "25-34", 35:  "35-44", 45:  "45-49", 50:  "50-55", 56:  "56+"}

- Occupation is chosen from the following choices: {0:  "other" or "not specified", 1:  "academic/educator", 2:  "artist", 3:  "clerical/admin", 4:  "college/grad student", 5:  "customer service", 6:  "doctor/health care", 7:  "executive/managerial", 8:  "farmer", 9:  "homemaker", 10:  "K-12 student", 11:  "lawyer", 12:  "programmer", 13:  "retired", 14:  "sales/marketing", 15:  "scientist", 16:  "self-employed", 17:  "technician/engineer", 18:  "tradesman/craftsman", 19:  "unemployed", 20:  "writer"}

In [4]:
ratings = pd.read_csv('./cleaned_datasets/ratings.dat', 
                 delimiter='::', 
                 encoding='latin1', 
                 engine = 'python', 
                 header = None,
                 names = ['user_id', 'movie_id', 'rating', 'timestamp']
)
movies = pd.read_csv('./cleaned_datasets/movies.dat', 
                 delimiter='::', 
                 encoding = 'latin1',
                 header = None,
                 engine = 'python',
                 names = ['movie_id', 'title', 'genres']
)
users = pd.read_csv('./cleaned_datasets/users.dat', 
                 delimiter='::', 
                 encoding='latin1', 
                 engine = 'python', 
                 header = None,
                 names = ['user_id', 'gender', 'age', 'occupation', 'zip_code']
)

In [5]:
from datetime import datetime

# TODO: later change to hive.sql
ratings['timestamp'] = ratings['timestamp'].apply(lambda timestamp: datetime.fromtimestamp(timestamp))

ratings = ratings.sort_values(['rating', 'timestamp'], ascending = [False, False])
ratings.head()

,user_id,movie_id,rating,timestamp
825438,4958,1407,5,2003-02-28 09:47:23
984025,5948,1180,5,2003-02-28 05:11:19
984632,5950,3196,5,2003-02-27 09:54:22
984849,5950,111,5,2003-02-27 09:50:41
984825,5950,2329,5,2003-02-27 09:45:48


In [6]:
movies.head()

,movie_id,title,genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [7]:
users.head()

,user_id,gender,age,occupation,zip_code
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455


## Create merged ratings and analysis

In [8]:
# merge datasets
merged = pd.merge(ratings, users, how = 'inner', on = 'user_id')
merged = pd.merge(merged, movies, how = 'inner', on = 'movie_id')
merged.head()

ratings_cleaned = merged[['user_id', 'movie_id', 'rating']].sort_values(by=['user_id', 'movie_id', 'rating'])
print(ratings_cleaned.head())

print(f"Unique number of users in the dataset: {ratings_cleaned['user_id'].nunique()}")
print(f"Unique number of movies in the dataset: {ratings_cleaned['movie_id'].nunique()}")

        user_id  movie_id  rating
171933        1         1       5
911414        1        48       5
21420         1       150       5
49104         1       260       4
34221         1       527       5
Unique number of users in the dataset: 6040
Unique number of movies in the dataset: 3706


#### Some movies in the movie dataset does not have any ratings.
These movies will not be used as input data.

In [9]:
# hence there are some movies missing in merged dataset
ratings[ratings['movie_id'] == 51]

,user_id,movie_id,rating,timestamp


In [ ]:
missing_movies = movies[~movies['movie_id'].isin(ratings_cleaned['movie_id'].unique())]
missing_movies

## Train Test Split

### Train test split

In [12]:
from sklearn.model_selection import train_test_split

# Split the data into training and testing sets
train, test = train_test_split(ratings_cleaned, test_size=0.0001, random_state=42)

# train = dataset w/o both verification & test
train, validation = train_test_split(train, test_size=0.001, random_state=42)

print(f"Unique number of users in the train dataset: {train['user_id'].nunique()}")
print(f"Unique number of movies in the train dataset: {train['movie_id'].nunique()}")

print(f"Unique number of users in the train dataset: {validation['user_id'].nunique()}")
print(f"Unique number of movies in the train dataset: {validation['movie_id'].nunique()}")

print(f"Unique number of users in the train dataset: {test['user_id'].nunique()}")
print(f"Unique number of movies in the train dataset: {test['movie_id'].nunique()}")

train

Unique number of users in the train dataset: 6040
Unique number of movies in the train dataset: 3706
Unique number of users in the train dataset: 812
Unique number of movies in the train dataset: 724
Unique number of users in the train dataset: 98
Unique number of movies in the train dataset: 95


,user_id,movie_id,rating
730373,4168,3082,3
627609,4284,2763,4
993682,798,2559,5
266328,4345,2529,3
412503,984,3099,4
...,...,...,...
841987,1799,533,1
749605,1203,1377,4
606134,5047,431,3
694799,735,3863,3


### Export train, validation, test

In [18]:
train.to_csv("cleaned_datasets/train.csv", index=False)
validation.to_csv("cleaned_datasets/validation.csv", index=False)
test.to_csv("cleaned_datasets/test.csv", index=False)